$$ \newcommand{\ket}[1]{\left|{#1}\right\rangle} $$
$$ \newcommand{\bra}[1]{\left\langle{#1}\right|} $$

# Quantum Phase Estimation
Assume we have a unitary operator $U$ with an eigenvector $\ket{u}$ and corresponding unknown eigenvalue $e^{2\pi i \phi}$. The goal of phase estimation is to estimate the value of $\phi$. 

#### Motivation
Phase estimation is an important subroutine in many known quantum algorithms, such as Shor's algorithm. In quantum physics it can estimate the eigenvalue spectrum of an arbitrary Hamiltonian to good precision with improved scaling to currently known classical algorithms. However, it requires long coherence time and thus is not "fit" for near-term quantum computers.

### Components
Makes use of the inverse quantum Fourier transform. In addition it uses hadamard gates and controlled-$U$ operations, where $U$ is the operator for which we wish to estimate the eigenvalues.

\begin{equation}
    H = \frac{1}{\sqrt{2}}
    \begin{bmatrix}
        1 & 1 \\
        1 & -1
    \end{bmatrix}
\end{equation}

The algorithm makes use of two quantum registers, or sets of qubits. The first contains $t$ qubits, we call these work qubits, and represents the precision we want for the eigenvalues due to the binary fraction representing the eigenvalues

$$
0.\phi_1 \phi_2 \dots \phi_t = \frac{\phi_1}{2^1} + \frac{\phi_1}{2^2} + \dots +\frac{\phi_t}{2^t}
$$

The second register contains $n$ qubits, we call these simulation qubits, depending on the operator $U$. 

### Algorithm
The work qubits is first put in superposition, by applying the hadamard gate to each qubit. Next we notice that by applying the operator $U$ multiple time, e.g. two times, we get

$$
U^2\ket{u} = e^{2\pi i \phi}U\ket{u} = e^{2\pi i (2\phi)}\ket{u}
$$

So by doing this $2^j$ number of times we get

$$
U^{2^j}\ket{u} = e^{2\pi i (2^j\phi)}\ket{u}
$$


After the initialization of the work qubits we apply a controlled-$U^{2^j}$ operation on the simulation qubits controlled by each work qubit, which then will be in the state

$$
\frac{1}{2^{t-1}}
\left(\ket{0} + e^{2\pi i (2^{t-1}\phi)}\ket{1}\right)
\left(\ket{0} + e^{2\pi i (2^{t-2}\phi)}\ket{1}\right)
\dots
\left(\ket{0} + e^{2\pi i (2^0\phi)}\ket{1}\right)
$$

By applying the inverse quantum Fourier transform to the work qubits we can measure it and get an estimate for $\phi$.

In [24]:
import qiskit as qk
import numpy as np
qk.IBMQ.load_account()
        
def PhaseEstimation(U,n,t,Ansatz=None):
    """ ________________
        
        Phase Estimation
        ________________
        
        Input: 
        
            Qcircuit = [qc,qr,cr,n]
                - U -> operator to estimate eigenvalues
                - n -> Number of qubits in second register
                - t -> Precision of eigenvalues, number of qubits in first register
        
        Output:
            
            Phase -> Spectrum of eigenvalues
    """
    qr = qk.QuantumRegister(t+n+2)
    cr = qk.ClassicalRegister(t+n+2)
    qc = qk.QuantumCircuit(qr,cr)
    #U.set_Qcircuit(qc,qr,n,t)
    dt = 0.1
    
    # Initialize / Create superposition
    for i in range(t):
        qc.h(qr[i])
    
    if Ansatz != None:
        qc,qr = Ansatz(qc,qr,n,t)
    else:
        for i in range(n):
            qc.h(qr[t+i])
        
    # Apply controlled-U operations
    for i in range(t):
        for j in range(2**i):
            #qc = U(qc,qr,cntrl=i,skip=t)
            qc = U(qc,qr,i,t,n,dt,h=1,Emax=500)
            
    # Inverse Quantum Fourier Transform | qc,qr,cr,_ = QFT([qc,qr,cr,t],inverse=True)
    for i in range(int(t/2)):
            qc.swap(qr[i],qr[-(i+1)])   
    for i in range(t):
        for j in range(i):
            qc.cu1(-np.pi/2**(i-j+1),qr[j],qr[i])
        qc.h(qr[i])
        
    # Measurement
    qc.measure(qr,cr)
    job = qk.execute(qc, backend = qk.Aer.get_backend('qasm_simulator'), shots=1024)
    result = job.result().get_counts(qc)
    print(reisult)
    measurements = []
    Emax = 500
    for key,value in result.items():
        key_ = key[n+1:]
        eigenstate = key[1:(n+1)]
        eigenstate = eigenstate[::-1]
        decimal = 0
        for i,bit in enumerate(key_):
            decimal += int(bit)*2**(-i-1)
        if value != 0:
            measurements.append(np.array([eigenstate, Emax-decimal*2*np.pi/t, value]))

    measurements = np.array(measurements)
    
    return measurements

In [25]:
#### Example BlackBox class #####

class BlackBox:
    def __init__(self,circuit):
        self.circuit = circuit
        self.set_Qcircuit(0,0,0)
    
    def set_Qcircuit(self,qc,qr,n,t=0):
        self.qc = qc
        self.qr = qr
        self.n = n
        self.t = t
        
    def __call__(self,qc,qr,cntrl=-1,skip=0):
        #qr=self.qr
        #assert self.qc != None
        #assert self.qr != None
        for i,circ_list in enumerate(self.circuit):
            
            for qbit in range(circ_list.size):
                operation = circ_list.get(qbit).op
                if operation == '': # If identity
                    continue
                if cntrl >= 0:
                    exec('qc.c{}(qr[{}],qr[{}])'.format(operation,qbit+skip,cntrl))
                else:
                    exec('qc.{}(qr[{}])'.format(operation,qbit+skip))
            
        return qc

In [26]:
def blackbox(qc,qr,cntrl,t,n,dt,h=1,Emax=500):
    #qr = qk.QuantumRegister(n+1)
    #cr = qk.ClassicalRegister(n+1)
    #qc = qk.QuantumCircuit(qr,cr)
    
    for i in range(t,t+n-1):
        # Fourth sum
        qc.cx(qr[i],qr[t+n])
        qc.cx(qr[i+1],qr[t+n])
        qc.crz(dt,qr[cntrl],qr[t+n])
        qc.cx(qr[i+1],qr[t+n])
        qc.cx(qr[i],qr[t+n])
    for i in range(t,t+n-1):
        # Third sum
        qc.cx(qr[i],qr[t+n])
        qc.cx(qr[i+1],qr[t+n])
        qc.rx(-np.pi/2,qr[i+1])
        qc.rx(-np.pi/2,qr[i]) 
        qc.crz(dt,qr[cntrl],qr[t+n])
        qc.rx(np.pi/2,qr[i+1])
        qc.rx(np.pi/2,qr[i])
        qc.cx(qr[i+1],qr[t+n])
        qc.cx(qr[i],qr[t+n])
    for i in range(t,t+n-1):
        # Second sum
        qc.cx(qr[i],qr[t+n])
        qc.cx(qr[i+1],qr[t+n])
        qc.ry(-np.pi/2,qr[i+1])
        qc.ry(-np.pi/2,qr[i]) 
        qc.crz(dt,qr[cntrl],qr[t+n])
        qc.ry(np.pi/2,qr[i+1])
        qc.ry(np.pi/2,qr[i])
        qc.cx(qr[i+1],qr[t+n])
        qc.cx(qr[i],qr[t+n])
    for i in range(t,t+n-1):
        # First sum
        qc.crz(2*h*dt,qr[cntrl],qr[i])
    
    qc.rz(Emax,qr[0])
    
    return qc

In [27]:
mes = PhaseEstimation(blackbox,4,4)

{'0111001101': 1, '1110000111': 4, '0110001111': 1, '0011011010': 3, '1100011000': 2, '0010000100': 4, '0010111010': 4, '1011001001': 2, '1110111011': 3, '1101101011': 3, '1001010011': 1, '1010111010': 2, '1110010001': 4, '1000100010': 3, '1010010011': 4, '1010110000': 2, '0001000110': 2, '1000011001': 1, '1000000110': 3, '0001101010': 2, '1011111001': 1, '0000110100': 1, '0010001101': 2, '0011001000': 5, '0011111101': 1, '0010101010': 6, '0000100000': 2, '1011111011': 2, '1101110010': 2, '0111101010': 1, '0011001110': 3, '0110010001': 3, '1000000100': 2, '0111111010': 2, '0101011011': 1, '1111011010': 2, '0011111111': 1, '1000111000': 1, '1100110011': 2, '0100100010': 3, '1001010000': 1, '1101101010': 3, '0100100000': 3, '0100111111': 1, '1100100010': 4, '0001011010': 1, '1010111001': 1, '1000110010': 6, '0011111001': 1, '1101111010': 1, '1010101010': 2, '1110100011': 2, '0001110010': 4, '0000100011': 1, '0010110000': 1, '0001110011': 5, '1001110000': 3, '0101000001': 4, '1000000111':

In [28]:
print(mes)

[['0111' '499.3618639922396' '1']
 ['0011' '499.6563883035136' '4']
 ['0011' '499.2636892218149' '1']
 ...
 ['0110' '499.4109513774519' '1']
 ['0100' '499.6563883035136' '1']
 ['1101' '499.1655144513902' '3']]


In [31]:
from operators import *
n_qubits = 4
n_fermi = 2
delta = 1
g = 1
g /= 4

h_pq = np.identity(n_qubits)
for p in range(n_qubits):
    h_pq[p,p] *= delta*(p - (p%2))/2

h_pqrs = np.zeros((n_qubits,n_qubits,n_qubits,n_qubits))
for p in range(0,n_qubits-1,2):
    for r in range(0,n_qubits-1,2):
        h_pqrs[p,p+1,r,r+1] = -0.5*g

Pairing = Hamiltonian(n_qubits)

circuit_list = Pairing.get_circuits(h_pq,h_pqrs)
for oplist in circuit_list:
    print(oplist.factor)
    for i in range(n_qubits):
        print('qb[{}]'.format(i),oplist.get(i).op)
        
U = BlackBox(circuit_list)
res = PhaseEstimation(U,20,n_qubits)


(0.75+0j)
qb[0] 
qb[1] 
qb[2] 
qb[3] 
(-0.125+0j)
qb[0] z
qb[1] 
qb[2] 
qb[3] 
(-0.125+0j)
qb[0] 
qb[1] z
qb[2] 
qb[3] 
(-0.125+0j)
qb[0] z
qb[1] z
qb[2] 
qb[3] 
(-0.0625+0j)
qb[0] x
qb[1] x
qb[2] x
qb[3] x
(0.0625-0j)
qb[0] x
qb[1] x
qb[2] y
qb[3] y
(-0.0625+0j)
qb[0] x
qb[1] y
qb[2] x
qb[3] y
(-0.0625+0j)
qb[0] x
qb[1] y
qb[2] y
qb[3] x
(-0.0625+0j)
qb[0] y
qb[1] x
qb[2] x
qb[3] y
(-0.0625+0j)
qb[0] y
qb[1] x
qb[2] y
qb[3] x
(0.0625-0j)
qb[0] y
qb[1] y
qb[2] x
qb[3] x
(-0.0625+0j)
qb[0] y
qb[1] y
qb[2] y
qb[3] y
(0.375+0j)
qb[0] 
qb[1] 
qb[2] z
qb[3] 
(0.375+0j)
qb[0] 
qb[1] 
qb[2] 
qb[3] z
(-0.125+0j)
qb[0] 
qb[1] 
qb[2] z
qb[3] z


In [29]:
print(res)

[['01010010011000110001' '500.0' '1']
 ['00001000100011101110' '500.0' '1']
 ['00101011100000111011' '500.0' '1']
 ...
 ['01010100000000010100' '500.0' '1']
 ['01110100011111101111' '500.0' '1']
 ['00101101010101001010' '500.0' '1']]


In [ ]:
#Pairing
H0 = [
      
]